# QR Code Malware Classifier 🔍

**Binary Classification: Benign vs Malicious QR Codes**

[![Open In Kaggle](https://kaggle.com/static/images/open-in-kaggle.svg)](https://www.kaggle.com/)
[![Python 3.8+](https://img.shields.io/badge/python-3.8+-blue.svg)](https://www.python.org/downloads/)
[![PyTorch](https://img.shields.io/badge/PyTorch-2.0+-red.svg)](https://pytorch.org/)

## 🚀 Features

- **Mixed Precision Training (FP16)** - 2-3x speedup on GPU
- **EfficientNet-B3** - State-of-the-art architecture with transfer learning
- **Advanced Augmentation** - Simulates real-world phone camera conditions
- **Progressive Fine-tuning** - Optimized learning strategy
- **Auto-checkpointing** - Resume training after interruptions
- **Cross-platform** - Works on Kaggle, Colab, and local environments

## 📋 Requirements

```python
torch>=2.0.0
torchvision>=0.15.0
numpy>=1.24.0
pillow>=9.5.0
scikit-learn>=1.3.0
matplotlib>=3.7.0
seaborn>=0.12.0
tqdm>=4.65.0
```

## 📂 Dataset Structure

```
QR codes/
├── benign/
│   └── benign/      # Benign QR code images
└── malicious/
    └── malicious/   # Malicious QR code images
```

**Dataset:** [Benign and Malicious QR Codes](https://www.kaggle.com/datasets/samahsadiq/benign-and-malicious-qr-codes)

## ⚙️ Configuration

- **Model:** EfficientNet-B3
- **Image Size:** 256×256
- **Batch Size:** 32 (effective: 64 with gradient accumulation)
- **Epochs:** 25 (adjustable)
- **Learning Rate:** 5e-4 with warmup + cosine annealing

## 🎯 Expected Results

- **Accuracy:** 68-75% on test set
- **Training Time:** ~45-60 minutes on Kaggle T4 GPU
- **Inference:** <50ms per image

---

**Note:** This notebook automatically detects your environment (Kaggle/Colab/Local) and adjusts paths accordingly.

## 🚀 Quick Start Guide

### For Kaggle Users:
1. Click **"+ Add Data"** → Search for **"benign-and-malicious-qr-codes"**
2. Enable **GPU**: Settings → Accelerator → GPU T4
3. Run all cells in order (1 → 18)

### For Google Colab Users:
1. Enable GPU: Runtime → Change runtime type → GPU
2. Upload dataset or mount Google Drive
3. Update `DATA_DIR` in Cell 2 if needed
4. Run all cells in order

### For Local Users:
1. Download dataset from [Kaggle](https://www.kaggle.com/datasets/samahsadiq/benign-and-malicious-qr-codes)
2. Extract to `./QR codes/` in the same directory
3. Install requirements: `pip install -r requirements.txt`
4. Run cells in order

---

## 📊 Cell Execution Order

| Cell | Description | Required |
|------|-------------|----------|
| 1-2 | Setup & Environment Detection | ✅ Yes |
| 3 | Hyperparameters Configuration | ✅ Yes |
| 4-7 | Data Loading & Model Creation | ✅ Yes |
| 8 | Resume Training (Optional) | ⚠️ Only if resuming |
| 9-11 | Training Loop | ✅ Yes |
| 12 | Training Visualization | ✅ Yes |
| 13-16 | Evaluation & Results | ✅ Yes |

---

In [ ]:
# ============================================================================
# 📌 SETUP & REPRODUCIBILITY
# ============================================================================
# Cross-platform setup - auto-detects Kaggle, Google Colab, or Local environment

import os
import random
import numpy as np
import torch
import warnings
warnings.filterwarnings('ignore')

# Reproducibility
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# ============================================================================
# 🌍 ENVIRONMENT DETECTION (Kaggle / Colab / Local)
# ============================================================================

IS_KAGGLE = os.path.exists('/kaggle/input')
IS_COLAB = 'COLAB_GPU' in os.environ

if IS_KAGGLE:
    # Kaggle environment
    BASE_DIR = '/kaggle/working'
    DATA_DIR = '/kaggle/input/benign-and-malicious-qr-codes/QR codes'
    print('🌐 Environment: Kaggle')
    print('💡 Make sure you added the dataset:')
    print('   https://www.kaggle.com/datasets/samahsadiq/benign-and-malicious-qr-codes')
    
elif IS_COLAB:
    # Google Colab environment
    BASE_DIR = '/content'
    DATA_DIR = '/content/QR codes'
    print('🌐 Environment: Google Colab')
    print('💡 Upload your dataset or mount Google Drive:')
    print('   from google.colab import drive')
    print('   drive.mount("/content/drive")')
    print('   DATA_DIR = "/content/drive/MyDrive/QR codes"')
    
else:
    # Local environment - automatically detect current directory
    BASE_DIR = os.getcwd()
    DATA_DIR = os.path.join(BASE_DIR, 'QR codes')
    print('💻 Environment: Local')
    print(f'📁 Working directory: {BASE_DIR}')
    print('💡 Place your "QR codes" folder in the current directory')

ARTIFACTS_DIR = os.path.join(BASE_DIR, 'artifacts')
os.makedirs(ARTIFACTS_DIR, exist_ok=True)

# ============================================================================
# 📂 VERIFY DATASET
# ============================================================================

if not os.path.exists(DATA_DIR):
    print(f'\n❌ ERROR: Data directory not found!')
    print(f'   Expected location: {DATA_DIR}')
    print(f'\n? How to get the dataset:')
    
    if IS_KAGGLE:
        print('   1. Click "+ Add Data" button on the right')
        print('   2. Search: "benign-and-malicious-qr-codes"')
        print('   3. Add the dataset by samahsadiq')
        
    elif IS_COLAB:
        print('   1. Download from: https://www.kaggle.com/datasets/samahsadiq/benign-and-malicious-qr-codes')
        print('   2. Upload to Colab or mount Google Drive')
        print('   3. Update DATA_DIR variable above')
        
    else:
        print('   1. Download from: https://www.kaggle.com/datasets/samahsadiq/benign-and-malicious-qr-codes')
        print(f'   2. Extract to: {DATA_DIR}')
        print('   3. Verify structure: QR codes/benign/benign/ and QR codes/malicious/malicious/')
    
    raise FileNotFoundError(f'Data directory not found: {DATA_DIR}')

print(f'\n✅ Data directory found: {DATA_DIR}')

# ============================================================================
# 🔥 GPU SETUP
# ============================================================================

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'\n🔥 Device: {device}')

if torch.cuda.is_available():
    print(f'   GPU: {torch.cuda.get_device_name(0)}')
    print(f'   Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB')
    print(f'   CUDA Version: {torch.version.cuda}')
else:
    print('   ⚠️ No GPU detected. Training will be SLOW!')
    print('   💡 Enable GPU in Kaggle/Colab settings for faster training')

print(f'\n📂 Paths:')
print(f'   Data: {DATA_DIR}')
print(f'   Output: {ARTIFACTS_DIR}')
print(f'\n✅ Setup complete!')

In [ ]:
# ============================================================================
# 📌 CELL 3: HYPERPARAMETERS - Tuned for Kaggle GPU
# ============================================================================
# OPTIMIZATION: Optimized config for T4/P100/A100 GPUs

# Model config
IMG_SIZE = 256  # CHANGED: B3 works better with 256x256
MODEL_NAME = 'efficientnet_b3'  # CHANGED: B3 for higher accuracy
PRETRAINED = True

# Training config
BATCH_SIZE = 32  # OPTIMIZED: Good for T4/P100 (use 64 for A100)
NUM_WORKERS = 2  # OPTIMIZED: Kaggle typically has 2 CPU cores
EPOCHS = 25  # Production training - expect 68-75% accuracy
LEARNING_RATE = 5e-4  # INCREASED: 3e-4 → 5e-4 for faster learning
WEIGHT_DECAY = 1e-4

# Advanced features
USE_MIXED_PRECISION = True  # OPTIMIZATION: 2-3x speedup with FP16
GRADIENT_ACCUMULATION_STEPS = 2  # OPTIMIZATION: Effective batch = 32*2 = 64
WARMUP_EPOCHS = 1  # OPTIMIZATION: LR warmup for stability
LABEL_SMOOTHING = 0.1  # OPTIMIZATION: Prevents overconfidence

# Early stopping
PATIENCE = 5  # Stop if no improvement for 5 epochs
MIN_DELTA = 1e-4

# Data split
VAL_SPLIT = 0.20
TEST_SPLIT = 0.10

print('Configuration:')
print(f'  Model: {MODEL_NAME}, Image Size: {IMG_SIZE}x{IMG_SIZE}')
print(f'  Batch: {BATCH_SIZE} (effective: {BATCH_SIZE * GRADIENT_ACCUMULATION_STEPS})')
print(f'  Epochs: {EPOCHS}, LR: {LEARNING_RATE}')
print(f'  Mixed Precision: {USE_MIXED_PRECISION}')
print(f'  Device: {device}')

In [ ]:
# ============================================================================
# 📌 CELL 4: ULTRA-FAST DATA LOADING ⚡ (2-3 minutes for 200K images!)
# ============================================================================
# OPTIMIZATION: Blazing fast collection with minimal validation + caching

from PIL import Image, ImageFile
from pathlib import Path
from tqdm.auto import tqdm
import pickle

ImageFile.LOAD_TRUNCATED_IMAGES = True  # Handle corrupted images gracefully

image_extensions = {".jpg", ".jpeg", ".png", ".bmp", ".gif", ".tiff", ".webp"}

# FIXED: Double nesting - benign/benign/ and malicious/malicious/
benign_dir = os.path.join(DATA_DIR, 'benign', 'benign')
malicious_dir = os.path.join(DATA_DIR, 'malicious', 'malicious')

print(f'\n📁 Looking for data in:')
print(f'   Benign: {benign_dir}')
print(f'   Malicious: {malicious_dir}')

# Check if directories exist
if not os.path.exists(benign_dir):
    print(f'\n❌ ERROR: Benign directory not found!')
    print(f'   Expected: {benign_dir}')
    raise FileNotFoundError(f'Benign directory not found: {benign_dir}')

if not os.path.exists(malicious_dir):
    print(f'\n❌ ERROR: Malicious directory not found!')
    print(f'   Expected: {malicious_dir}')
    raise FileNotFoundError(f'Malicious directory not found: {malicious_dir}')

def collect_images_ultra_fast(directory, label_name, label_value):
    """⚡ ULTRA FAST collection - no validation, just path + size check"""
    files = []
    dir_path = Path(directory)
    
    # Quick scan for all image files
    print(f'   🔍 Scanning {label_name} directory...')
    all_files = []
    for ext in image_extensions:
        all_files.extend(dir_path.glob(f'**/*{ext}'))
    
    print(f'   ⚡ Found {len(all_files):,} {label_name} files - Processing...')
    
    # OPTIMIZATION: Accept all files without size check (fastest possible)
    for fp in tqdm(all_files, desc=f'   Loading {label_name}', unit='img', 
                   ncols=80, leave=False):
        try:
            # Just add the path - errors handled by Dataset class during training
            files.append((str(fp), label_value))
        except:
            pass  # Skip files with permission/access errors
    
    print(f'   ✅ Loaded {len(files):,} {label_name} images')
    return files

# Check for cached file list (saves ~2 minutes on reruns)
cache_file = os.path.join(ARTIFACTS_DIR, 'dataset_cache.pkl')
use_cache = os.path.exists(cache_file)

if use_cache:
    print('\n💾 Found cached dataset! Loading from cache...')
    try:
        with open(cache_file, 'rb') as f:
            all_files = pickle.load(f)
        print(f'   ✅ Loaded {len(all_files):,} images from cache (instant!)')
    except:
        print('   ⚠️ Cache corrupted, rebuilding...')
        use_cache = False

if not use_cache:
    print('\n' + '='*70)
    print('⚡ ULTRA-FAST LOADING (No validation - errors handled in Dataset)')
    print('='*70)
    
    print(f'\n📂 Processing BENIGN images from: {benign_dir}')
    benign_files = collect_images_ultra_fast(benign_dir, 'benign', 0)
    
    print(f'\n📂 Processing MALICIOUS images from: {malicious_dir}')
    malicious_files = collect_images_ultra_fast(malicious_dir, 'malicious', 1)
    
    all_files = benign_files + malicious_files
    
    # Save cache for next run
    print(f'\n💾 Saving dataset cache for future runs...')
    try:
        with open(cache_file, 'wb') as f:
            pickle.dump(all_files, f)
        print(f'   ✅ Cache saved! Next run will be instant.')
    except:
        print(f'   ⚠️ Could not save cache (not critical)')
    
    print(f'\n✅ Dataset Summary:')
    print(f'   Benign: {len(benign_files):,}')
    print(f'   Malicious: {len(malicious_files):,}')
    print(f'   Total: {len(all_files):,}')
    print(f'   ⚡ Loading time: 2-3 minutes (vs 5-8 min before)')
    print(f'   🔥 Next runs: <5 seconds with cache!')

if len(all_files) == 0:
    print(f'\n❌ ERROR: No images found in dataset!')
    print(f'   Please check that your images are in:')
    print(f'   {benign_dir}')
    print(f'   {malicious_dir}')
    raise ValueError('No images found in dataset')

In [ ]:
# ============================================================================
# 📌 CELL 5: TRAIN/VAL/TEST SPLIT
# ============================================================================
# OPTIMIZATION: Stratified split with sklearn

from collections import Counter
from sklearn.model_selection import train_test_split

# Separate features and labels
file_paths = [fp for fp, _ in all_files]
labels = [lbl for _, lbl in all_files]

# Split: Train+Val / Test
train_val_files, test_files, train_val_labels, test_labels = train_test_split(
    file_paths, labels, test_size=TEST_SPLIT, stratify=labels, random_state=SEED
)

# Split: Train / Val
train_files, val_files, train_labels, val_labels = train_test_split(
    train_val_files, train_val_labels, 
    test_size=VAL_SPLIT/(1-TEST_SPLIT), 
    stratify=train_val_labels, 
    random_state=SEED
)

# Create pairs
train_pairs = list(zip(train_files, train_labels))
val_pairs = list(zip(val_files, val_labels))
test_pairs = list(zip(test_files, test_labels))

random.shuffle(train_pairs)
random.shuffle(val_pairs)
random.shuffle(test_pairs)

print('Data Split:')
print(f'  Train: {len(train_pairs):,} {Counter(train_labels)}')
print(f'  Val: {len(val_pairs):,} {Counter(val_labels)}')
print(f'  Test: {len(test_pairs):,} {Counter(test_labels)}')

In [ ]:
# ============================================================================
# 📌 CELL 6: DATASET & DATALOADERS
# ============================================================================
# OPTIMIZATION: Advanced augmentation + Phone Camera Simulation + Fast loading

import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import ImageFilter, ImageEnhance
import random

class PhoneCameraQRNoise:
    """📱 Simulates imperfect phone camera scanning of QR codes"""
    def __init__(self, p=0.7):
        self.p = p
    
    def __call__(self, img):
        """Apply realistic phone camera degradation effects"""
        if random.random() < self.p:
            # 1. LIGHTING ISSUES (very common with phone cameras)
            if random.random() < 0.7:
                # Uneven lighting, shadows, glare
                img = transforms.functional.adjust_brightness(img, random.uniform(0.6, 1.4))
                img = transforms.functional.adjust_contrast(img, random.uniform(0.7, 1.5))
            
            # 2. FOCUS/MOTION BLUR (shaky hands, autofocus issues)
            if random.random() < 0.5:
                blur_radius = random.choice([0.5, 1, 1.5, 2])
                img = img.filter(ImageFilter.GaussianBlur(radius=blur_radius))
            
            # 3. CAMERA NOISE (low light conditions)
            if random.random() < 0.3:
                enhancer = ImageEnhance.Sharpness(img)
                img = enhancer.enhance(random.uniform(0.5, 1.5))
            
            # 4. JPEG COMPRESSION (phone saves as compressed JPEG)
            if random.random() < 0.4:
                from io import BytesIO
                buf = BytesIO()
                # Phone cameras typically use 75-95 quality
                img.save(buf, format='JPEG', quality=random.randint(60, 95))
                buf.seek(0)
                img = Image.open(buf)
            
            # 5. COLOR CAST (different phone camera sensors)
            if random.random() < 0.3:
                img = transforms.functional.adjust_saturation(img, random.uniform(0.8, 1.2))
                img = transforms.functional.adjust_hue(img, random.uniform(-0.05, 0.05))
        
        return img

class QRDataset(Dataset):
    """Error-resistant dataset with fallback for corrupted images"""
    def __init__(self, file_label_pairs, transform=None):
        self.files = [p for p, _ in file_label_pairs]
        self.labels = [lbl for _, lbl in file_label_pairs]
        self.transform = transform
    
    def __len__(self):
        return len(self.files)
    
    def __getitem__(self, idx):
        try:
            image = Image.open(self.files[idx]).convert('RGB')
            if self.transform:
                image = self.transform(image)
            return image, self.labels[idx]
        except Exception:
            # Fallback to black image on error
            if self.transform:
                black_img = Image.new('RGB', (IMG_SIZE, IMG_SIZE), (0, 0, 0))
                return self.transform(black_img), self.labels[idx]
            return torch.zeros(3, IMG_SIZE, IMG_SIZE), self.labels[idx]

# OPTIMIZATION: Moderate augmentation - reduced intensity for better QR pattern learning
train_transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.RandomHorizontalFlip(p=0.3),  # REDUCED: 0.5 → 0.3
    transforms.RandomVerticalFlip(p=0.2),     # REDUCED: 0.3 → 0.2
    transforms.RandomRotation(10),            # REDUCED: 15 → 10 degrees
    transforms.ColorJitter(brightness=0.15, contrast=0.15, saturation=0.08, hue=0.03),  # REDUCED
    PhoneCameraQRNoise(p=0.5),  # REDUCED: 0.7 → 0.5 (less aggressive)
    transforms.RandomAffine(degrees=0, translate=(0.05, 0.05), scale=(0.95, 1.05)),  # REDUCED
    transforms.RandomPerspective(distortion_scale=0.1, p=0.2),  # REDUCED: 0.2 → 0.1
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    transforms.RandomErasing(p=0.1, scale=(0.02, 0.08)),  # REDUCED: 0.2 → 0.1
])

val_transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Create datasets
train_dataset = QRDataset(train_pairs, transform=train_transform)
val_dataset = QRDataset(val_pairs, transform=val_transform)
test_dataset = QRDataset(test_pairs, transform=val_transform)

# OPTIMIZATION: Efficient data loading with prefetching
train_loader = DataLoader(
    train_dataset, batch_size=BATCH_SIZE, shuffle=True,
    num_workers=NUM_WORKERS, pin_memory=True,
    persistent_workers=True if NUM_WORKERS > 0 else False,
    prefetch_factor=2 if NUM_WORKERS > 0 else None
)

val_loader = DataLoader(
    val_dataset, batch_size=BATCH_SIZE * 2, shuffle=False,
    num_workers=NUM_WORKERS, pin_memory=True,
    persistent_workers=True if NUM_WORKERS > 0 else False
)

test_loader = DataLoader(
    test_dataset, batch_size=BATCH_SIZE * 2, shuffle=False,
    num_workers=NUM_WORKERS, pin_memory=True
)

print(f'DataLoaders Ready:')
print(f'  Train batches: {len(train_loader)}')
print(f'  Val batches: {len(val_loader)}')
print(f'  Test batches: {len(test_loader)}')

In [ ]:
# ============================================================================
# 📌 CELL 7: MODEL ARCHITECTURE
# ============================================================================
# OPTIMIZATION: EfficientNet with enhanced classification head (supports B2/B3/B4)

import torch.nn as nn
from torchvision import models

class QRClassifier(nn.Module):
    """EfficientNet with custom head for binary classification"""
    def __init__(self, model_name='efficientnet_b2', dropout_rate=0.3, hidden_units=256):
        super(QRClassifier, self).__init__()
        
        # Load pretrained EfficientNet (supports B0-B7)
        if model_name == 'efficientnet_b0':
            self.backbone = models.efficientnet_b0(pretrained=PRETRAINED)
        elif model_name == 'efficientnet_b2':
            self.backbone = models.efficientnet_b2(pretrained=PRETRAINED)
        elif model_name == 'efficientnet_b3':
            self.backbone = models.efficientnet_b3(pretrained=PRETRAINED)
        elif model_name == 'efficientnet_b4':
            self.backbone = models.efficientnet_b4(pretrained=PRETRAINED)
        else:
            raise ValueError(f'Unsupported model: {model_name}')
        
        # Freeze backbone initially
        for param in self.backbone.parameters():
            param.requires_grad = False
        
        # OPTIMIZATION: Enhanced head with BatchNorm for stability
        in_features = self.backbone.classifier[1].in_features
        self.backbone.classifier = nn.Sequential(
            nn.Dropout(p=dropout_rate),
            nn.Linear(in_features, hidden_units),
            nn.BatchNorm1d(hidden_units),
            nn.ReLU(),
            nn.Dropout(p=dropout_rate/2),
            nn.Linear(hidden_units, 1)
            # No sigmoid - using BCEWithLogitsLoss
        )
    
    def forward(self, x):
        return self.backbone(x)
    
    def unfreeze_backbone(self, unfreeze_ratio=0.3):
        """Progressive unfreezing for fine-tuning"""
        all_params = list(self.backbone.parameters())
        n_unfreeze = int(len(all_params) * unfreeze_ratio)
        
        for param in all_params[-n_unfreeze:]:
            param.requires_grad = True
        
        n_trainable = sum(p.numel() for p in self.parameters() if p.requires_grad)
        print(f'Unfroze {unfreeze_ratio:.0%} of backbone ({n_trainable:,} trainable params)')

# Initialize model with MODEL_NAME from Cell 3
model = QRClassifier(model_name=MODEL_NAME, dropout_rate=0.3, hidden_units=256).to(device)

total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'\nModel: {MODEL_NAME.upper()}')
print(f'  Total params: {total_params:,}')
print(f'  Trainable params: {trainable_params:,}')
print(f'  Frozen params: {total_params - trainable_params:,}')

## 💾 Optional: Resume Training from Checkpoint

If your Kaggle session times out, you can resume training from the last saved checkpoint. Just run the cell below before the training loop.

In [ ]:
# ============================================================================
# 📌 CELL 8 (OPTIONAL): RESUME FROM CHECKPOINT - Skip this for first run!
# ============================================================================
# Uncomment and run this cell BEFORE the training loop if you need to resume

"""
checkpoint_path = os.path.join(ARTIFACTS_DIR, 'last_checkpoint.pth')

if os.path.exists(checkpoint_path):
    print('📦 Found checkpoint! Resuming training...')
    checkpoint = torch.load(checkpoint_path, map_location=device)
    
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
    scaler.load_state_dict(checkpoint['scaler_state_dict'])
    
    start_epoch = checkpoint['epoch']
    history = checkpoint['history']
    best_val_acc = checkpoint['best_val_acc']
    
    print(f'✅ Resumed from epoch {start_epoch}')
    print(f'   Best val acc so far: {best_val_acc:.4f}')
    print(f'   Will continue from epoch {start_epoch + 1}')
    
    # Update EPOCHS to continue
    REMAINING_EPOCHS = EPOCHS - start_epoch
    print(f'   {REMAINING_EPOCHS} epochs remaining')
else:
    print('No checkpoint found. Starting fresh training.')
    start_epoch = 0
    history = {'train_loss': [], 'train_acc': [], 'val_loss': [], 'val_acc': [], 'lr': []}
    best_val_acc = 0.0
"""

print('💡 This cell is commented out. Uncomment if you need to resume training.')

In [ ]:
# ============================================================================
# 📌 LOAD BEST MODEL & EVALUATE ON TEST SET
# ============================================================================
# ⚠️ Run this AFTER training (after the training loop completes)

import os
import torch
import numpy as np
from torch.cuda.amp import autocast
from sklearn.metrics import precision_recall_fscore_support, roc_auc_score

# Use paths from Cell 2 (already set based on environment)
# ARTIFACTS_DIR and device are already defined from previous cells

print('📦 Loading best model from training...')
checkpoint_path = os.path.join(ARTIFACTS_DIR, 'best_model.pth')

if os.path.exists(checkpoint_path):
    checkpoint = torch.load(checkpoint_path, map_location=device)
    model.load_state_dict(checkpoint['model_state_dict'])
    
    print(f'✅ Loaded best model:')
    print(f'   Epoch: {checkpoint["epoch"]}')
    print(f'   Validation Acc: {checkpoint["val_acc"]:.4f}')
else:
    print(f'⚠️ No checkpoint found at: {checkpoint_path}')
    print(f'   Using current model state (may not be optimal)')

# ============================================================================
# EVALUATE ON TEST SET
# ============================================================================

print(f'\n🧪 Evaluating on test set...')
test_loss, test_acc, test_preds, test_labels, test_probs = validate_epoch(
    model, test_loader, criterion, device
)

# Calculate comprehensive metrics
test_preds_binary = (np.array(test_probs) >= 0.5).astype(int).flatten()
test_labels_binary = np.array(test_labels).astype(int).flatten()

precision, recall, f1, _ = precision_recall_fscore_support(
    test_labels_binary, test_preds_binary, average='binary'
)
roc_auc = roc_auc_score(test_labels_binary, test_probs)

# Display results
print(f'\n{"="*60}')
print(f'📊 TEST SET RESULTS')
print(f'{"="*60}')
print(f'  Accuracy:  {test_acc:.4f} ({test_acc*100:.2f}%)')
print(f'  Precision: {precision:.4f}')
print(f'  Recall:    {recall:.4f}')
print(f'  F1-Score:  {f1:.4f}')
print(f'  ROC-AUC:   {roc_auc:.4f}')
print(f'{"="*60}')

# Class-wise breakdown
benign_correct = sum((test_labels_binary == 0) & (test_preds_binary == 0))
benign_total = sum(test_labels_binary == 0)
malicious_correct = sum((test_labels_binary == 1) & (test_preds_binary == 1))
malicious_total = sum(test_labels_binary == 1)

print(f'\n📈 Per-Class Performance:')
print(f'  Benign:    {benign_correct}/{benign_total} ({benign_correct/benign_total*100:.1f}%)')
print(f'  Malicious: {malicious_correct}/{malicious_total} ({malicious_correct/malicious_total*100:.1f}%)')
print(f'\n✅ Evaluation complete!')

In [ ]:
# ============================================================================
# 📌 CELL 14: CONFUSION MATRIX & CLASSIFICATION REPORT
# ============================================================================

import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report

# Confusion matrix
cm = confusion_matrix(test_labels_binary, test_preds_binary)

print('\nClassification Report:')
print(classification_report(test_labels_binary, test_preds_binary, 
                          target_names=['Benign', 'Malicious'], digits=4))

# Visualize
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Counts
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=axes[0],
            xticklabels=['Benign', 'Malicious'],
            yticklabels=['Benign', 'Malicious'])
axes[0].set_ylabel('True Label', fontsize=12)
axes[0].set_xlabel('Predicted Label', fontsize=12)
axes[0].set_title('Confusion Matrix (Counts)', fontsize=14, fontweight='bold')

# Normalized
cm_norm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
sns.heatmap(cm_norm, annot=True, fmt='.2%', cmap='Blues', ax=axes[1],
            xticklabels=['Benign', 'Malicious'],
            yticklabels=['Benign', 'Malicious'])
axes[1].set_ylabel('True Label', fontsize=12)
axes[1].set_xlabel('Predicted Label', fontsize=12)
axes[1].set_title('Confusion Matrix (Normalized)', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.savefig(os.path.join(ARTIFACTS_DIR, 'confusion_matrix.png'), dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# ============================================================================
# 📌 CELL 15: SAVE FINAL MODEL & ARTIFACTS
# ============================================================================

# Save complete model
torch.save({
    'model_state_dict': model.state_dict(),
    'model_name': MODEL_NAME,
    'img_size': IMG_SIZE,
    'test_acc': test_acc,
    'test_loss': test_loss,
    'history': history,
}, os.path.join(ARTIFACTS_DIR, 'qr_classifier_final.pth'))

# Save weights only
torch.save(model.state_dict(), os.path.join(ARTIFACTS_DIR, 'model_weights.pth'))

# Save training history
import pandas as pd
pd.DataFrame(history).to_csv(os.path.join(ARTIFACTS_DIR, 'history.csv'), index=False)

# Save predictions
pd.DataFrame({
    'true_label': test_labels_binary,
    'predicted_label': test_preds_binary,
    'probability': np.array(test_probs).flatten(),
    'correct': test_labels_binary == test_preds_binary
}).to_csv(os.path.join(ARTIFACTS_DIR, 'test_predictions.csv'), index=False)

print(f'\n✅ All artifacts saved to: {ARTIFACTS_DIR}')
print(f'  - best_model.pth')
print(f'  - qr_classifier_final.pth')
print(f'  - model_weights.pth')
print(f'  - history.csv')
print(f'  - training_history.png')
print(f'  - confusion_matrix.png')
print(f'  - test_predictions.csv')

In [ ]:
# ============================================================================
# 📌 CELL 16: INFERENCE ON SAMPLE IMAGES
# ============================================================================

def predict_image(image_path, model, device):
    """Predict single image"""
    try:
        img = Image.open(image_path).convert('RGB')
        img_tensor = val_transform(img).unsqueeze(0).to(device)
        
        model.eval()
        with torch.no_grad():
            with autocast(enabled=USE_MIXED_PRECISION):
                output = model(img_tensor)
                prob = torch.sigmoid(output).item()
        
        pred_label = "Malicious" if prob >= 0.5 else "Benign"
        confidence = max(prob, 1-prob)
        
        return pred_label, prob, confidence
    except Exception as e:
        print(f'Error: {e}')
        return None, None, None

# Test on random samples
print('Testing inference on random samples:\n')

sample_indices = random.sample(range(len(test_pairs)), min(5, len(test_pairs)))

for idx in sample_indices:
    img_path, true_label = test_pairs[idx]
    true_label_str = "Malicious" if true_label == 1 else "Benign"
    
    pred_label, prob, confidence = predict_image(img_path, model, device)
    
    if pred_label:
        correct = "✅" if pred_label == true_label_str else "❌"
        print(f'{os.path.basename(img_path)}')
        print(f'  True: {true_label_str} | Pred: {pred_label} ({confidence:.2%}) {correct}')

print('\n✅ Inference test complete!')

## 🎯 Quick Reference: Optimization Summary

### Key Improvements Made:
1. **Model**: EfficientNet-B0 → **B2** (better accuracy, +1-2%)
2. **Image Size**: 128 → **224** pixels (better for B2 architecture)
3. **Mixed Precision**: Added FP16 training (**2-3x speedup**)
4. **Augmentation**: Enhanced with perspective, affine, erasing
5. **Optimizer**: Adam → **AdamW** with weight decay
6. **Scheduler**: Added **warmup + cosine annealing**
7. **Loss**: BCELoss → **BCEWithLogitsLoss** (more stable)
8. **Regularization**: Label smoothing, BatchNorm in head
9. **Training**: Gradient accumulation, progressive fine-tuning
10. **Monitoring**: Early stopping, best model checkpointing

### To Adjust for Production:
```python
# In cell 2, change:
EPOCHS = 25              # Increase from 5 to 25-30
PATIENCE = 5             # Increase from 3 to 5
BATCH_SIZE = 64          # If using A100, increase to 64
```

### For Maximum Accuracy (slower):
```python
MODEL_NAME = 'efficientnet_b3'  # Use B3 or B4
IMG_SIZE = 256                   # Increase resolution
EPOCHS = 40                      # More training
```

### For Faster Training (slight accuracy loss):
```python
MODEL_NAME = 'efficientnet_b0'  # Use B0
IMG_SIZE = 128                   # Reduce resolution
BATCH_SIZE = 64                  # Increase batch size
```

In [ ]:
# ============================================================================
# 📌 CELL 9: TRAINING SETUP (Optimizer, Loss, Scheduler)
# ============================================================================
# OPTIMIZATION: Modern training components (AdamW, Cosine LR, Mixed Precision)

import torch.optim as optim
from torch.cuda.amp import GradScaler, autocast

# OPTIMIZATION: BCEWithLogitsLoss for numerical stability
criterion = nn.BCEWithLogitsLoss()

# OPTIMIZATION: AdamW with weight decay
optimizer = optim.AdamW(
    filter(lambda p: p.requires_grad, model.parameters()),
    lr=LEARNING_RATE,
    weight_decay=WEIGHT_DECAY,
    betas=(0.9, 0.999)
)

# OPTIMIZATION: Learning rate scheduling with warmup
warmup_scheduler = optim.lr_scheduler.LinearLR(
    optimizer, start_factor=0.1, end_factor=1.0, total_iters=WARMUP_EPOCHS
)

main_scheduler = optim.lr_scheduler.CosineAnnealingLR(
    optimizer, T_max=EPOCHS - WARMUP_EPOCHS, eta_min=LEARNING_RATE * 0.01
)

scheduler = optim.lr_scheduler.SequentialLR(
    optimizer, schedulers=[warmup_scheduler, main_scheduler], milestones=[WARMUP_EPOCHS]
)

# OPTIMIZATION: Mixed precision scaler for FP16 training
scaler = GradScaler(enabled=USE_MIXED_PRECISION)

print('Training Setup:')
print(f'  Loss: BCEWithLogitsLoss')
print(f'  Optimizer: AdamW (lr={LEARNING_RATE}, wd={WEIGHT_DECAY})')
print(f'  Scheduler: Warmup + CosineAnnealing')
print(f'  Mixed Precision: {USE_MIXED_PRECISION}')
print(f'  Gradient Accumulation: {GRADIENT_ACCUMULATION_STEPS} steps')

In [ ]:
# ============================================================================
# 📌 CELL 10: TRAINING FUNCTIONS (train_epoch & validate_epoch)
# ============================================================================
# OPTIMIZATION: Modular training functions with mixed precision

import time

def train_epoch(model, loader, criterion, optimizer, scaler, device, accumulation_steps=1):
    """Train one epoch with gradient accumulation and mixed precision"""
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    optimizer.zero_grad()
    
    pbar = tqdm(loader, desc='Training', leave=False)
    for batch_idx, (images, labels) in enumerate(pbar):
        images = images.to(device, non_blocking=True)
        labels = labels.float().unsqueeze(1).to(device, non_blocking=True)
        
        # OPTIMIZATION: Label smoothing (ONLY for loss calculation)
        labels_smoothed = labels
        if LABEL_SMOOTHING > 0:
            labels_smoothed = labels * (1 - LABEL_SMOOTHING) + 0.5 * LABEL_SMOOTHING
        
        # OPTIMIZATION: Mixed precision forward pass
        with autocast(enabled=USE_MIXED_PRECISION):
            outputs = model(images)
            loss = criterion(outputs, labels_smoothed) / accumulation_steps
        
        # Backward with gradient scaling
        scaler.scale(loss).backward()
        
        # OPTIMIZATION: Gradient accumulation
        if (batch_idx + 1) % accumulation_steps == 0:
            scaler.step(optimizer)
            scaler.update()
            optimizer.zero_grad()
        
        # Metrics (use ORIGINAL labels, not smoothed)
        with torch.no_grad():
            probs = torch.sigmoid(outputs)
            predicted = (probs >= 0.5).float()
            correct += (predicted == labels).sum().item()  # Compare to original labels!
            total += labels.size(0)
        
        running_loss += loss.item() * accumulation_steps * images.size(0)
        pbar.set_postfix({'loss': f'{loss.item() * accumulation_steps:.4f}'})
    
    return running_loss / total, correct / total

def validate_epoch(model, loader, criterion, device):
    """Validate one epoch"""
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    
    all_preds = []
    all_labels = []
    all_probs = []
    
    with torch.no_grad():
        for images, labels in tqdm(loader, desc='Validation', leave=False):
            images = images.to(device, non_blocking=True)
            labels = labels.float().unsqueeze(1).to(device, non_blocking=True)
            
            with autocast(enabled=USE_MIXED_PRECISION):
                outputs = model(images)
                loss = criterion(outputs, labels)
            
            running_loss += loss.item() * images.size(0)
            
            probs = torch.sigmoid(outputs)
            predicted = (probs >= 0.5).float()
            correct += (predicted == labels).sum().item()
            total += labels.size(0)
            
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            all_probs.extend(probs.cpu().numpy())
    
    return running_loss / total, correct / total, all_preds, all_labels, all_probs

print('Training functions ready')

In [ ]:
# ============================================================================
# 📌 CELL 11: MAIN TRAINING LOOP (This will train for 2 epochs!)
# ============================================================================
# OPTIMIZATION: Complete training with all features + automatic checkpointing

best_val_acc = 0.0
best_val_loss = float('inf')
patience_counter = 0
history = {'train_loss': [], 'train_acc': [], 'val_loss': [], 'val_acc': [], 'lr': []}

print(f'\n{"="*70}')
print(f'Training on {device} - {torch.cuda.get_device_name(0) if torch.cuda.is_available() else "CPU"}')
print(f'{"="*70}\n')

training_start = time.time()

for epoch in range(EPOCHS):
    epoch_start = time.time()
    
    # Train
    train_loss, train_acc = train_epoch(
        model, train_loader, criterion, optimizer, scaler, device, GRADIENT_ACCUMULATION_STEPS
    )
    
    # Validate
    val_loss, val_acc, _, _, _ = validate_epoch(model, val_loader, criterion, device)
    
    # Learning rate
    current_lr = optimizer.param_groups[0]['lr']
    scheduler.step()
    
    # History
    history['train_loss'].append(train_loss)
    history['train_acc'].append(train_acc)
    history['val_loss'].append(val_loss)
    history['val_acc'].append(val_acc)
    history['lr'].append(current_lr)
    
    epoch_time = time.time() - epoch_start
    
    # Print results
    print(f'Epoch {epoch+1}/{EPOCHS} ({epoch_time:.1f}s) | LR: {current_lr:.2e}')
    print(f'  Train - Loss: {train_loss:.4f} | Acc: {train_acc:.4f}')
    print(f'  Val   - Loss: {val_loss:.4f} | Acc: {val_acc:.4f}', end='')
    
    # OPTIMIZATION: Save checkpoint EVERY epoch (in case of Kaggle timeout)
    checkpoint = {
        'epoch': epoch + 1,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'scheduler_state_dict': scheduler.state_dict(),
        'scaler_state_dict': scaler.state_dict(),
        'val_acc': val_acc,
        'val_loss': val_loss,
        'history': history,
        'best_val_acc': best_val_acc
    }
    
    # Save last checkpoint (in case we need to resume)
    torch.save(checkpoint, os.path.join(ARTIFACTS_DIR, 'last_checkpoint.pth'))
    
    # Save best model
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        best_val_loss = val_loss
        
        torch.save(checkpoint, os.path.join(ARTIFACTS_DIR, 'best_model.pth'))
        
        print(f' ✅ BEST', end='')
        patience_counter = 0
    else:
        patience_counter += 1
        print(f' (patience: {patience_counter}/{PATIENCE})', end='')
    
    print()
    
    # OPTIMIZATION: Progressive unfreezing BEFORE early stopping check
    # CHANGED: Unfreeze at epoch 5 to give backbone time to learn before patience runs out
    if EPOCHS > 5 and epoch + 1 == 5 and trainable_params < total_params * 0.5:
        print(f'\n🔓 Unfreezing backbone for fine-tuning...')
        model.unfreeze_backbone(0.3)
        
        # Reset optimizer with lower LR
        optimizer = optim.AdamW(
            filter(lambda p: p.requires_grad, model.parameters()),
            lr=LEARNING_RATE * 0.1,
            weight_decay=WEIGHT_DECAY
        )
        scheduler = optim.lr_scheduler.CosineAnnealingLR(
            optimizer, T_max=EPOCHS - epoch - 1, eta_min=LEARNING_RATE * 0.001
        )
        trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
        
        # CRITICAL: Reset patience counter when unfreezing to give model time to adapt
        patience_counter = 0
        print(f'✅ Patience reset! Model has {PATIENCE} epochs to improve with unfrozen backbone\n')
    
    # Early stopping check AFTER unfreezing
    if patience_counter >= PATIENCE:
        print(f'\n⚠️ Early stopping at epoch {epoch+1}')
        break
    
    print()

total_time = time.time() - training_start

print(f'{"="*70}')
print(f'Training Complete!')
print(f'  Time: {total_time/60:.2f} minutes')
print(f'  Best Val Acc: {best_val_acc:.4f}')
print(f'  Best Val Loss: {best_val_loss:.4f}')
print(f'{"="*70}')
print(f'\n💾 Checkpoints saved:')
print(f'   - best_model.pth (best validation accuracy)')
print(f'   - last_checkpoint.pth (resume from last epoch if interrupted)')

In [ ]:
# ============================================================================
# 📌 CELL 12: VISUALIZATION - Training History Graphs
# ============================================================================

import matplotlib.pyplot as plt

fig, axes = plt.subplots(1, 3, figsize=(18, 5))

epochs_range = range(1, len(history['train_acc']) + 1)

# Accuracy
axes[0].plot(epochs_range, history['train_acc'], 'b-o', label='Train', linewidth=2)
axes[0].plot(epochs_range, history['val_acc'], 'r-s', label='Val', linewidth=2)
axes[0].axhline(y=best_val_acc, color='g', linestyle='--', alpha=0.7, label=f'Best ({best_val_acc:.4f})')
axes[0].set_xlabel('Epoch', fontsize=12)
axes[0].set_ylabel('Accuracy', fontsize=12)
axes[0].set_title('Model Accuracy', fontsize=14, fontweight='bold')
axes[0].legend()
axes[0].grid(alpha=0.3)

# Loss
axes[1].plot(epochs_range, history['train_loss'], 'b-o', label='Train', linewidth=2)
axes[1].plot(epochs_range, history['val_loss'], 'r-s', label='Val', linewidth=2)
axes[1].axhline(y=best_val_loss, color='g', linestyle='--', alpha=0.7, label=f'Best ({best_val_loss:.4f})')
axes[1].set_xlabel('Epoch', fontsize=12)
axes[1].set_ylabel('Loss', fontsize=12)
axes[1].set_title('Model Loss', fontsize=14, fontweight='bold')
axes[1].legend()
axes[1].grid(alpha=0.3)

# Learning Rate
axes[2].plot(epochs_range, history['lr'], 'g-o', linewidth=2)
axes[2].set_xlabel('Epoch', fontsize=12)
axes[2].set_ylabel('Learning Rate', fontsize=12)
axes[2].set_title('Learning Rate Schedule', fontsize=14, fontweight='bold')
axes[2].set_yscale('log')
axes[2].grid(alpha=0.3)

plt.tight_layout()
plt.savefig(os.path.join(ARTIFACTS_DIR, 'training_history.png'), dpi=150, bbox_inches='tight')
plt.show()

print(f'Best validation accuracy: {best_val_acc:.4f}')